In [2]:
# hier ist die testumgebung für zahlen
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from chronos import ChronosPipeline

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-large",
  device_map="cpu",
  torch_dtype=torch.bfloat16,
)


In [14]:
# we want a plot with percent in range for 
import random
# daten
import os, json
wd = "/home/a2/code/fin/tradingbot/data/stocks"
sxsx = os.listdir(wd)

store = [] # content: {"name", "len", "prediction"}
from tqdm import tqdm
prediction_length = 25
import plotly.graph_objects as go
prediction_length = 25
count_arr = [0 for _ in range(25)]
count_5 = 0
boolean_count_5d_return = 0
for s in tqdm(sxsx):
    p = os.path.join(wd, s)
    df = pd.read_csv(p)
    size = len(df)
    if size < 100:
        continue
    t_main = torch.tensor(df["Close"])[:-random.randint(1, size//2)] # here we try to randomize days to minimize makro
    context = t_main[:-prediction_length]


    if (len(context) < 90):
        continue

    forecast = pipeline.predict(context, prediction_length)


    new_dict = {"name": s, "ctxt_len": len(context), "prediction": forecast.tolist()}
    store.append(new_dict)

    low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
    r = t_main[-prediction_length:] # real data
    # perc of in 80% interval for 25 time steps
    b_in_interval = True
    # 5d acc
    last_val = context[-1] # referenzwert
    fifth_day_interval = (low[4], high[4])
    respective_returns = ((fifth_day_interval[0]-last_val)/last_val, (fifth_day_interval[1]-last_val)/last_val)
    real_return = (t_main[-21]-last_val)/last_val
    if real_return>respective_returns[0] and real_return<respective_returns[1]:
        count_5 += 1
    if (abs(respective_returns[0])-abs(respective_returns[1]) < 0):
        if real_return > 0:
            boolean_count_5d_return += 1
    else:
        if real_return<0:
            boolean_count_5d_return += 1

    for n, i in enumerate(r):
        if not (i > low[n] and i < high[n]):
            b_in_interval = False
            break
    if b_in_interval:
        count_arr[n] += 1
    
# maybe also look at 80 perc interval and try to put a 5 day acuracy
    # plot this shit 
my_length = len(sxsx)
print(f"after 5 days in interval: {count_5/my_length}")
print(f"after 5 days right estimate of up/down: {boolean_count_5d_return/my_length}")
count_arr = list(map(lambda x: x/my_length, count_arr))
fig = go.Figure()
fig.add_trace(go.Scatter(y=count_arr, mode='lines+markers', name='80 perc interval acc'))
fig.show()

# 0.12
# save data

with open("predictions.json", "w") as f:
    json.dump(store, f)

  0%|          | 0/5884 [00:00<?, ?it/s]

100%|██████████| 5884/5884 [4:30:14<00:00,  2.76s/it]  

after 5 days in interval: 0.663494221617947
after 5 days right estimate of up/down: 0.48759347382732837


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [15]:

with open("predictions.json", "w") as f:
    json.dump(store, f)

In [6]:
df = pd.read_csv(p)

In [8]:
len(df)

256

In [9]:
a = [0,0,0,0,0,0]

In [10]:
a[-0]

0